In [1]:
import sys
import os
import time
import scanpy as sc
import warnings
warnings.filterwarnings("ignore")

sys.path.append('/public/workspace/ruru_97/projects/schgnn/csMAHN')
import utils.preprocess as pp
from utils.utility import *
from utils.train import Trainer

import importlib as im
im.reload(pp)


Bad key "text.kerning_factor" on line 4 in
/public/workspace/ruru_97/anaconda3/envs/schgnn/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


<module 'utils.preprocess' from '/public/workspace/ruru_97/projects/schgnn/csMAHN/utils/preprocess.py'>

In [2]:
reverse=False
aligned=True
tissue="pancreas"
gse_ids=["GSE84113", "GSE84113"]
species=('human', 'mouse')
dsnames=('human', 'mouse')
path_data='/public/workspace/ruru_97/projects/data'
resdir='/public/workspace/ruru_97/projects/schgnn/result'
homo_method='biomart'
key_class='cell_type'
n_hvgs=2000
n_degs=50

seed = 123
stages=[200, 200, 200]
nfeats=64  # enbedding size #128
hidden=64 # 128
input_drop=0.2
att_drop=0.2
residual=True

threshold=0.9 # 0.8
lr=0.01  # lr = 0.01
weight_decay=0.001
patience=100
enhance_gama=10
simi_gama=0.1 

In [3]:
# reverse reference to query, query to reference.
if reverse:
    gse_ids = gse_ids[::-1]
    species = species[::-1]
    dsnames = dsnames[::-1]
seed_all(seed)
path_homo = f'{path_data}/homo/{homo_method}/input/{species[0]}_to_{species[1]}.txt'
path_specie_1 = f'{path_data}/ByTissue/{tissue}/{gse_ids[0]}/input/{dsnames[0]}.h5ad'
path_specie_2 = f'{path_data}/ByTissue/{tissue}/{gse_ids[1]}/input/{dsnames[1]}.h5ad'

# make file to save
time_tag = make_nowtime_tag()
curdir = f'{resdir}/{tissue}/{tissue}-{gse_ids[0]}_{dsnames[0]}-{gse_ids[1]}_{dsnames[1]}-{time_tag}'
model_dir = os.path.join(curdir, 'model_')
figdir = os.path.join(curdir, 'fig_')
os.mkdir(curdir)
os.mkdir(figdir)
os.mkdir(model_dir)
checkpt_file = model_dir + "/mutistages"
print(checkpt_file)

for i in range(len(stages)):
    res_dir = os.path.join(curdir, f'res_{i}')
    os.mkdir(res_dir)
homo = pd.read_csv(path_homo)
adata_species_1 = sc.read_h5ad(path_specie_1)
adata_species_2 = sc.read_h5ad(path_specie_2)

if aligned:
    adata_species_1, adata_species_2 = pp.aligned_type([adata_species_1, adata_species_2], 'cell_type')
else:
    pp.get_type_counts_info(adata_species_1, adata_species_2)

/public/workspace/ruru_97/projects/schgnn/result/pancreas/pancreas-GSE84113_human-GSE84113_mouse-03-14-13.28.07/model_/mutistages
----raw----
                    reference  query
alpha                    2326    191
beta                     2525    894
delta                     601    218
ductal                   1077    275
endothelial               252    139
gamma                     255     41
macrophage                 55     36
quiescent_stellate        173     47
----new----
                    reference  query
alpha                    2326    191
beta                     2525    894
delta                     601    218
ductal                   1077    275
endothelial               252    139
gamma                     255     41
macrophage                 55     36
quiescent_stellate        173     47


In [4]:
print(
    f'Task: refernece:{gse_ids[0]}_{dsnames[0]} {adata_species_1.shape[0]} cells x {adata_species_1.shape[1]} gene -> query:{gse_ids[1]}_{dsnames[1]} {adata_species_2.shape[0]} cells x {adata_species_2.shape[1]} gene in {tissue}')

Task: refernece:GSE84113_human 7264 cells x 20125 gene -> query:GSE84113_mouse 1841 cells x 14878 gene in pancreas


In [5]:
start = time.time()
# knn时间较长
adatas, features_genes, nodes_genes, scnets, one2one, n2n = pp.process_for_graph([adata_species_1, adata_species_2],
                                                                                    homo,
                                                                                    key_class,
                                                                                    'leiden',
                                                                                    n_hvgs=n_hvgs,
                                                                                    n_degs=n_degs)
g, inter_net, one2one_gene_nodes_net, cell_label, n_classes, list_idx = pp.make_graph(adatas,
                                                                                        aligned,
                                                                                        key_class,
                                                                                        features_genes,
                                                                                        nodes_genes,
                                                                                        scnets,
                                                                                        one2one,
                                                                                        n2n,
                                                                                        has_mnn=True,
                                                                                        seed=seed)
end = time.time()
# 包括预处理时间
print('Times preprocess for graph:{:.2f}'.format(end - start))

Homolog information follows
ortholog_one2one      16612
ortholog_many2many     5356
ortholog_one2many      3174
Name: Mouse homology type, dtype: int64


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


the time2 of processing adatas is 204.9959225654602
Leiden results:
0    515
1    276
2    274
3    256
4    222
5    137
6     76
7     48
8     37
Name: leiden, dtype: int64
the time of leiden is 1.2902448177337646
the time of degs is 23.64616823196411
--------------hvgs, degs info---------------
num of reference_hvgs,reference_degs,reference_higs are 2000,353,2125
num of query_hvgs,query_degs,query_higs are 2000,389,2195
--------------gene nodes info---------------
num of reference_gene_node is 3305
num of query_gene_node is 3037
--------------homo edges---------------
ortholog_one2one      2508
ortholog_one2many      194
ortholog_many2many     194
Name: Mouse homology type, dtype: int64
--------------homo edges---------------
ortholog_one2one    2508
Name: Mouse homology type, dtype: int64
knn time is 7.228848934173584 s
mnn time is 0.005603313446044922 s
the time of compute mnn is  7.78513503074646 s
Inter-net pairs ACC: 0.9758064516129032
-------------------nodes info------------

In [17]:
import utils.train as train1
im.reload(train1)

<module 'utils.train' from '/public/workspace/ruru_97/projects/schgnn/csMAHN/utils/train.py'>

In [20]:
trainer = train1.Trainer(adatas,
                    g,
                    inter_net,
                    list_idx,
                    cell_label,
                    n_classes,
                    key_class,

                    threshold=threshold)
out = trainer.train(curdir=curdir, 
            checkpt_file=checkpt_file,
            stages=[200,200,200],
            nfeats=nfeats,
            hidden=hidden,
            enhance_gama=enhance_gama,
            simi_gama=simi_gama)

MYHGNN(
  (prelu): PReLU(num_parameters=1)
  (dropout): Dropout(p=0.5, inplace=False)
  (input_drop): Dropout(p=0.2, inplace=False)
  (cell_embedings): ParameterDict(
      (C): Parameter containing: [torch.FloatTensor of size 503x64]
      (CC): Parameter containing: [torch.FloatTensor of size 503x64]
      (CCGGC): Parameter containing: [torch.FloatTensor of size 503x64]
      (CGGC): Parameter containing: [torch.FloatTensor of size 503x64]
      (CGGCC): Parameter containing: [torch.FloatTensor of size 503x64]
      (CCGGCC): Parameter containing: [torch.FloatTensor of size 503x64]
  )
  (cell_project_layers): Sequential(
    (0): Conv1d1x1()
    (1): LayerNorm((6, 64), eps=1e-05, elementwise_affine=True)
    (2): PReLU(num_parameters=1)
    (3): Dropout(p=0.5, inplace=False)
    (4): Conv1d1x1()
    (5): LayerNorm((6, 64), eps=1e-05, elementwise_affine=True)
    (6): PReLU(num_parameters=1)
    (7): Dropout(p=0.5, inplace=False)
  )
  (cell_semantic_aggr_layers): Transformer(
    (

In [12]:
trainer.get_confuse_matrix(is_probability=True)

pre_label,alpha,beta,delta,ductal,endothelial,gamma,macrophage,quiescent_stellate
true_label,,,,,,,,
alpha,0.973822,0.010471,0.010471,0.005236,0.0,0.000000,0.0,0.0
beta,0.006711,0.986577,0.006711,0.000000,0.0,0.000000,0.0,0.0
delta,0.013761,0.004587,0.977064,0.000000,0.0,0.004587,0.0,0.0
ductal,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.0,0.0
endothelial,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.0,0.0
gamma,0.731707,0.000000,0.000000,0.000000,0.0,0.268293,0.0,0.0
macrophage,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.0,0.0
quiescent_stellate,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1.0
